In [1]:
import requests
import bs4
import pandas as pd
import re

In [2]:
# URL della pagina
url_province = 'https://italia.indettaglio.it/ita/province/province.html'
url_regioni = 'https://italia.indettaglio.it/ita/regioni/regioni.html'

# Eseguire la richiesta HTTP
requisicao = requests.get(url_province, verify=False)
pagina_province = bs4.BeautifulSoup(requisicao.text, "html.parser")

# Eseguire la richiesta HTTP
requisicao = requests.get(url_regioni, verify=False)
pagina_regioni = bs4.BeautifulSoup(requisicao.text, "html.parser")

c:\Users\schit\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'italia.indettaglio.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\schit\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'italia.indettaglio.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [26]:
regioni_italiane = {
    "Piemonte": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Valle d'Aosta": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Lombardia": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Liguria": {"macro_area": "Nord", "dettagliata": "Nord-Ovest"},
    "Veneto": {"macro_area": "Nord", "dettagliata": "Nord-Est"},
    "Trentino-Alto Adige": {"macro_area": "Nord", "dettagliata": "Nord-Est"},
    "Friuli Venezia Giulia": {"macro_area": "Nord", "dettagliata": "Nord-Est"},
    "Emilia-Romagna": {"macro_area": "Nord", "dettagliata": "Nord"},
    "Toscana": {"macro_area": "Centro", "dettagliata": "Centro"},
    "Umbria": {"macro_area": "Centro", "dettagliata": "Centro"},
    "Marche": {"macro_area": "Centro", "dettagliata": "Centro-Est"},
    "Lazio": {"macro_area": ["Centro", "Sud"], "dettagliata": ["Centro", "Centro-Sud"]},  # Lazio ha due posizioni
    "Abruzzo": {"macro_area": "Sud", "dettagliata": "Centro-Sud"},
    "Molise": {"macro_area": "Sud", "dettagliata": "Centro-Sud"},
    "Campania": {"macro_area": "Sud", "dettagliata": "Sud"},
    "Puglia": {"macro_area": "Sud", "dettagliata": "Sud-Est"},
    "Basilicata": {"macro_area": "Sud", "dettagliata": "Sud"},
    "Calabria": {"macro_area": "Sud", "dettagliata": "Sud"},
    "Sicilia": {"macro_area": "Sud", "dettagliata": "Sud-Ovest"},
    "Sardegna": {"macro_area": "Ovest", "dettagliata": "Ovest"}
}

print(regioni_italiane)

{'Piemonte': {'macro_area': 'Nord', 'dettagliata': 'Nord-Ovest'}, "Valle d'Aosta": {'macro_area': 'Nord', 'dettagliata': 'Nord-Ovest'}, 'Lombardia': {'macro_area': 'Nord', 'dettagliata': 'Nord-Ovest'}, 'Liguria': {'macro_area': 'Nord', 'dettagliata': 'Nord-Ovest'}, 'Veneto': {'macro_area': 'Nord', 'dettagliata': 'Nord-Est'}, 'Trentino-Alto Adige': {'macro_area': 'Nord', 'dettagliata': 'Nord-Est'}, 'Friuli Venezia Giulia': {'macro_area': 'Nord', 'dettagliata': 'Nord-Est'}, 'Emilia-Romagna': {'macro_area': 'Nord', 'dettagliata': 'Nord'}, 'Toscana': {'macro_area': 'Centro', 'dettagliata': 'Centro'}, 'Umbria': {'macro_area': 'Centro', 'dettagliata': 'Centro'}, 'Marche': {'macro_area': 'Centro', 'dettagliata': 'Centro-Est'}, 'Lazio': {'macro_area': ['Centro', 'Sud'], 'dettagliata': ['Centro', 'Centro-Sud']}, 'Abruzzo': {'macro_area': 'Sud', 'dettagliata': 'Centro-Sud'}, 'Molise': {'macro_area': 'Sud', 'dettagliata': 'Centro-Sud'}, 'Campania': {'macro_area': 'Sud', 'dettagliata': 'Sud'}, 'Pu

In [33]:
# Trova la tabella
tabella = pagina_province.find("table", class_="table")

# Estrarre i dati
regioni = []
province = []

for riga in tabella.find_all("tr"):
    colonne = riga.find_all("td")
    if len(colonne) == 2:
        regione = colonne[0].text.strip()
        lista_province = [a.text.strip() for a in colonne[1].find_all("a")]

        for provincia in lista_province:
            regioni.append(str(regione).replace(":", ""))
            province.append(provincia)

# Creare un DataFrame con i dati
df = pd.DataFrame({"Regione": regioni, "Provincia": province})
print(df.shape[0])

# Stampare i primi risultati
regione_provincia = df
df.head()

107


,Regione,Provincia
0,Abruzzo,Chieti
1,Abruzzo,L'Aquila
2,Abruzzo,Pescara
3,Abruzzo,Teramo
4,Basilicata,Matera


In [35]:
informazioni_geografiche = pd.read_csv("Comuni.csv")
informazioni_geografiche.head()

,sigla_provincia,Comune,Provincia
0,TO,Agliè,Torino
1,TO,Airasca,Torino
2,TO,Ala di Stura,Torino
3,TO,Albiano d'Ivrea,Torino
4,TO,Almese,Torino


In [38]:
def imposta_regione(sigla):
    regione = regione_provincia.loc[regione_provincia["Provincia"] == sigla, "Regione"]
    
    # Se la Serie è vuota, restituisci un valore di default (es. "Sconosciuto" o None)
    return str(regione.iloc[0]) if not regione.empty else "Sconosciuto"

# Applica la funzione alla colonna "sigla_provincia"
informazioni_geografiche["Regione"] = informazioni_geografiche["Provincia"].apply(imposta_regione)

In [42]:
informazioni_geografiche[informazioni_geografiche["Comune"] == "Civita"]

,sigla_provincia,Comune,Provincia,Regione
6230,CS,Civita,Cosenza,Calabria


In [4]:
df['Regione'].value_counts()

Regione
Lombardia                12
Toscana                  10
Emilia-Romagna            9
Sicilia                   9
Piemonte                  8
Veneto                    7
Puglia                    6
Calabria                  5
Campania                  5
Lazio                     5
Marche                    5
Sardegna                  5
Abruzzo                   4
Liguria                   4
Friuli-Venezia Giulia     4
Basilicata                2
Trentino-Alto Adige       2
Umbria                    2
Molise                    2
Valle d'Aosta             1
Name: count, dtype: int64

In [5]:
df[df["Regione"] == "Lombardia"]

,Regione,Provincia
38,Lombardia,Bergamo
39,Lombardia,Brescia
40,Lombardia,Como
41,Lombardia,Cremona
42,Lombardia,Lecco
43,Lombardia,Lodi
44,Lombardia,Mantova
45,Lombardia,Milano
46,Lombardia,Monza e della Brianza
47,Lombardia,Pavia


In [6]:
# Trova tutte le tabelle nella pagina
tabelle = pagina_province.find_all("table", class_="table")

# Assicuriamoci di selezionare la tabella giusta (quella con 5 colonne)
tabella_popolazione = None
for tabella in tabelle:
    if len(tabella.find_all("tr")[0].find_all("td")) == 5:
        tabella_popolazione = tabella
        break

# Se la tabella non è stata trovata, interrompiamo
if tabella_popolazione is None:
    print("Tabella della popolazione non trovata.")
else:
    # Liste per salvare i dati
    province = []
    pop_maschile = []
    pop_femminile = []
    pop_totale = []
    ultracentenari = []

    # Itera sulle righe della tabella
    for riga in tabella_popolazione.find_all("tr")[1:]:  # Salta la prima riga (intestazioni)
        colonne = riga.find_all("td")
        if len(colonne) == 5:
            provincia = colonne[0].text.strip()
            maschi = colonne[1].text.strip().replace(".", "")
            femmine = colonne[2].text.strip().replace(".", "")
            totale = colonne[3].text.strip().replace(".", "")
            centenari = colonne[4].text.strip().replace(".", "")

            province.append(provincia)
            pop_maschile.append(int(maschi))
            pop_femminile.append(int(femmine))
            pop_totale.append(int(totale))
            ultracentenari.append(int(centenari))

    # Creare un DataFrame con i dati
    df = pd.DataFrame({
        "Provincia": province,
        "Popolazione Maschile": pop_maschile,
        "Popolazione Femminile": pop_femminile,
        "Popolazione Totale": pop_totale,
        "Ultracentenari": ultracentenari
    })

print(df.shape[0])
df.head()

107


,Provincia,Popolazione Maschile,Popolazione Femminile,Popolazione Totale,Ultracentenari
0,Agrigento,217412,230326,447738,107
1,Alessandria,208164,223721,431885,228
2,Ancona,230229,247663,477892,169
3,Aosta,62564,65734,128298,38
4,Arezzo,168004,178438,346442,126


In [7]:
# Trova tutte le tabelle nella pagina
tabelle = pagina_regioni.find_all("table", class_="table")

# Assicuriamoci di selezionare la tabella giusta (quella con 5 colonne)
tabella_popolazione = None
for tabella in tabelle:
    if len(tabella.find_all("tr")[0].find_all("td")) == 5:
        tabella_popolazione = tabella
        break

# Se la tabella non è stata trovata, interrompiamo
if tabella_popolazione is None:
    print("Tabella della popolazione non trovata.")
else:
    # Liste per salvare i dati
    province = []
    pop_maschile = []
    pop_femminile = []
    pop_totale = []
    ultracentenari = []

    # Itera sulle righe della tabella
    for riga in tabella_popolazione.find_all("tr")[1:]:  # Salta la prima riga (intestazioni)
        colonne = riga.find_all("td")
        if len(colonne) == 5:
            provincia = colonne[0].text.strip()
            maschi = colonne[1].text.strip().replace(".", "")
            femmine = colonne[2].text.strip().replace(".", "")
            totale = colonne[3].text.strip().replace(".", "")
            centenari = colonne[4].text.strip().replace(".", "")

            province.append(provincia)
            pop_maschile.append(int(maschi))
            pop_femminile.append(int(femmine))
            pop_totale.append(int(totale))
            ultracentenari.append(int(centenari))

    # Creare un DataFrame con i dati
    df = pd.DataFrame({
        "Provincia": province,
        "Popolazione Maschile": pop_maschile,
        "Popolazione Femminile": pop_femminile,
        "Popolazione Totale": pop_totale,
        "Ultracentenari": ultracentenari
    })

df.head()

,Provincia,Popolazione Maschile,Popolazione Femminile,Popolazione Totale,Ultracentenari
0,Abruzzo,647812,683762,1331574,467
1,Basilicata,282525,294094,576619,156
2,Calabria,964960,1011671,1976631,629
3,Campania,2848043,3013486,5861529,1120
4,Emilia-Romagna,2155222,2295286,4450508,1874


---

## Dati del comune

In [8]:
# URL della pagina
url_comune = 'https://italia.indettaglio.it/ita/calabria/civita.html'
#url_comune = 'https://italia.indettaglio.it/ita/lazio/roma.html'

# Imposta il comune
comune = "Civita"

# Eseguire la richiesta HTTP
requisicao = requests.get(url_comune, verify=False)
pagina_del_comune = bs4.BeautifulSoup(requisicao.text, "html.parser")

c:\Users\schit\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'italia.indettaglio.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [9]:
def pulisci_dataframe(data_dict, colonne_da_rinominare):
    """
    Pulisce un DataFrame disordinato, aggiungendo automaticamente la prima riga 
    e rinominando le colonne specificate.

    :param data_dict: Dizionario dei dati grezzi
    :param colonne_da_rinominare: Lista di nomi per le colonne finali
    :return: DataFrame pulito
    """
    
    # Convertiamo il dizionario in un DataFrame temporaneo
    df_temp = pd.DataFrame(data_dict)

    # Recuperiamo automaticamente la prima riga (nomi effettivi delle colonne)
    prima_riga = [col.replace(":", "").strip() for col in df_temp.columns[:len(colonne_da_rinominare)]]
    
    # Creiamo un nuovo DataFrame con la prima riga inclusa
    df_clean = pd.DataFrame({
        colonne_da_rinominare[i]: [prima_riga[i]] + df_temp.iloc[:, i].str.replace(":", "").str.strip().tolist()
        for i in range(len(colonne_da_rinominare))
    })

    # Pulizia e conversione dei numeri
    for col in df_clean.columns:
        if df_clean[col].str.contains("%").any():  # Se contiene percentuali
            df_clean[col] = df_clean[col].str.replace(",", ".").str.rstrip("%").astype(float) / 100
        elif df_clean[col].str.replace(".", "", regex=False).str.isnumeric().all():  # Se è un numero
            df_clean[col] = df_clean[col].str.replace(".", "", regex=False).astype(int)  # Rimuove i punti

    return df_clean

In [10]:
# Trova il paragrafo con le informazioni
dati_paragrafo = pagina_del_comune.find("p", class_="well")

if dati_paragrafo:
    testo = dati_paragrafo.get_text()

    # Estrai le informazioni con slicing testuale
    info = {}
    info["Municipio"] = testo.split("Municipio: ")[1].split(",")[0]
    info["Provincia"] = testo.split("Provincia: ")[1].split(".")[0]
    info["Distanza dal capoluogo"] = testo.split("Distanza dal capoluogo")[1].split(": ")[1].split(" ")[0] + " Km"
    info["Abitanti"] = testo.split("Abitanti: ")[1].split(" ")[0]
    info["Denominazione"] = testo.split("Denominazione: ")[1].split(".")[0]

    # Stampare il risultato
    for k, v in info.items():
        print(f"{k}: {v}")

else:
    print("Errore: Impossibile trovare il paragrafo con i dati richiesti.")

Municipio: Piazza Municipio
Provincia: Cosenza
Distanza dal capoluogo: 74 Km
Abitanti: 902
Denominazione: Civitesi


In [11]:
# Trova il paragrafo che contiene la densità abitativa
densita_text = None
for p in pagina_del_comune.find_all("p"):
    if "Densità abitativa" in p.text:
        densita_text = p.text
        break

# Estrai il numero dalla stringa trovata
if densita_text:
    match = re.search(r"Densità abitativa:\s*([0-9.,]+)", densita_text)
    if match:
        densita = match.group(1)
        print(f"Densità abitativa: {densita} abitanti per km²")
    else:
        print("Densità abitativa non trovata.")
else:
    print("Sezione sulla densità abitativa non trovata.")

Densità abitativa: 41,51 abitanti per km²


In [12]:
# Cerca tutte le <div> che contengono tabelle (le tabelle sono racchiuse in div con classe "tb_resp")
div_tables = pagina_del_comune.find_all("div", class_="tb_resp")

families_table = None
for div in div_tables:
    table = div.find("table")
    if table:
        # Verifica se la prima riga (con classe "info") contiene "Numero di componenti"
        header_row = table.find("tr", class_="info")
        if header_row:
            cells = header_row.find_all("td")
            if cells and "Numero di componenti" in cells[0].get_text():
                families_table = table
                break

if families_table:
    # Estrazione delle righe della tabella
    rows = []
    for tr in families_table.find_all("tr"):
        cols = [td.get_text(strip=True) for td in tr.find_all("td")]
        rows.append(cols)
    
    # La struttura della tabella è la seguente:
    # Prima riga (header): ["Numero di componenti", "1", "2", "3", "4", "5", "6 o più"]
    # Seconda riga (dati): ["Numero di famiglie", "391", "390", "244", "156", "50", "32"]
    header = rows[0]
    data = rows[1]
    
    # Creare il DataFrame con la prima riga come intestazione
    df = pd.DataFrame([data], columns=header)
    df.rename(columns={"6 o piÃ¹": "6 o più"}, inplace=True)
    
    print("Tabella 'Famiglie e loro numerosità di componenti':")
    #print(df)
    
    # Salva il risultato in un file CSV
    #df.to_csv("famiglie_solarolo.csv", index=False)
else:
    print("Tabella 'Famiglie e loro numerosità di componenti' non trovata.")

df.head()

Tabella 'Famiglie e loro numerosità di componenti':


,Numero di componenti,1,2,3,4,5,6 o più
0,Numero di famiglie,149,132,69,54,11,2


In [21]:
def table(title):
    # Trova il titolo della sezione desiderata
    titolo_sezione = pagina_del_comune.find("span", id="accatre", string=title)
    # Dati sulla popolazione straniera residente a Solarolo

    if titolo_sezione:
        # Trova la tabella che segue il titolo della sezione
        tabella = titolo_sezione.find_next("table", class_="table")

        if tabella:
            # Estrai le intestazioni
            intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

            # Estrai i dati della tabella
            dati = []
            for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
                colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
                dati.append(colonne)

            # Creazione del DataFrame
            df = pd.DataFrame(dati, columns=intestazioni)
            
            # Salva il DataFrame in un file CSV
            #df.to_csv("popolazione_straniera_solarolo.csv", index=False, encoding="utf-8")

            #print("File CSV creato con successo: popolazione_straniera_solarolo.csv")
        else:
            print("Errore: Impossibile trovare la tabella della popolazione straniera.")
    else:
        print("Errore: Impossibile trovare la sezione 'Dati sulla popolazione straniera residente a Solarolo'.")
    
    return df

topici = ["Geografia, Anagrafe e Statistica", f"Dati sulla popolazione residente a {comune}", f"Dati sulla popolazione straniera residente a {comune}",
          f"Livelli di scolarizzazione a {comune}"]

for x in topici:
    print(x)
    display(table(x))


Geografia, Anagrafe e Statistica


,Industrie:,10,Addetti:,38,Percentuale sul totale:,"22,62%"
0,Servizi:,20,Addetti:,45,Percentuale sul totale:,"26,79%"
1,Amministrazione:,4,Addetti:,24,Percentuale sul totale:,"14,29%"
2,Altro:,20,Addetti:,61,Percentuale sul totale:,"36,31%"


Dati sulla popolazione residente a Civita


,EtÃ,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,>74
0,Maschi,16,8,19,18,25,18,18,32,31,26,31,26,35,22,28,81
1,Femmine,12,15,18,20,17,23,20,24,19,28,34,33,40,18,39,109
2,Totale,28,23,37,38,42,41,38,56,50,54,65,59,75,40,67,190


Dati sulla popolazione straniera residente a Civita


,EtÃ,Maschi,Femmine,Totale
0,Da 0 a 29 anni,5,4,9
1,Da 30 a 54 anni,3,6,9
2,PiÃ¹ di 54 anni,0,2,2


Livelli di scolarizzazione a Civita


,Genere,Laurea,Diploma,Licenza Media,Licenza Elementare,Alfabeti,Analfabeti
0,Maschi,31,120,112,95,47,11
1,Femmine,41,107,75,133,74,25
2,Totale,72,227,187,228,121,36


In [25]:
def transform_dict_to_df(data_dict):
    df = pd.DataFrame(data_dict)
    new_data = {}

    for col in df.columns:
        if col != list(data_dict.keys())[0]:  
            for idx, age_group in df[list(data_dict.keys())[0]].items():
                new_col_name = f"{col}_{age_group}"
                new_data[new_col_name] = df.at[idx, col]  # Aggiungiamo il valore corrispondente

    # Creiamo il nuovo DataFrame con una sola riga
    new_df = pd.DataFrame([new_data])
    
    return new_df

data = transform_dict_to_df(table("Dati sulla popolazione straniera residente a Civita")
                     .rename(columns={"EtÃ": "Età"})
                     .to_dict())

for x in list(data.columns):
    nuovo = str(x).replace("PiÃ¹", "Più")
    data.rename(columns={x: nuovo}, inplace=True)  # Modifica in-place

data["Comune"] = comune
data

,Maschi_Da 0 a 29 anni,Maschi_Da 30 a 54 anni,Maschi_Più di 54 anni,Femmine_Da 0 a 29 anni,Femmine_Da 30 a 54 anni,Femmine_Più di 54 anni,Totale_Da 0 a 29 anni,Totale_Da 30 a 54 anni,Totale_Più di 54 anni,Comune
0,5,3,0,4,6,2,9,9,2,Civita


In [15]:
colonne_finali = ["Settore", "Numero_Aziende", "Addetti", "Percentuale_Totale"]
df_pulito = pulisci_dataframe(table("Geografia, Anagrafe e Statistica").to_dict(), colonne_finali)
df_pulito["Comune"] = comune
df_pulito

,Settore,Numero_Aziende,Addetti,Percentuale_Totale,Comune
0,Industrie,10,Addetti,38,Civita
1,Servizi,20,Addetti,45,Civita
2,Amministrazione,4,Addetti,24,Civita
3,Altro,20,Addetti,61,Civita


## Redditi e edifici

In [16]:
def correzione_moneta(valore):
    valore = str(valore).replace("â¬", "€")
    return valore

def table(title):
    # Trova il titolo della sezione desiderata
    titolo_sezione = pagina_del_comune.find("h4", string=title)

    if titolo_sezione:
        # Trova la tabella che segue il titolo della sezione
        tabella = titolo_sezione.find_next("table", class_="table")

        if tabella:
            # Estrai le intestazioni
            intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

            # Estrai i dati della tabella
            dati = []
            for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
                colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
                dati.append(colonne)

            # Creazione del DataFrame
            df = pd.DataFrame(dati, columns=intestazioni)
            df["Comune"] = comune

            if "Reddito" in df.columns:
                df["Reddito"] = df["Reddito"].apply(correzione_moneta)
                df["Media annuale"] = df["Media annuale"].apply(correzione_moneta)
                df["Media mensile"] = df["Media mensile"].apply(correzione_moneta)
                df["Anno precedente"] = df["Anno precedente"].apply(correzione_moneta)
                df["Variazione"] = df["Variazione"].apply(correzione_moneta)

            # Salva il DataFrame in un file CSV
            #df.to_csv("edifici_solarolo_costruzione.csv", index=False, encoding="utf-8")

            #print("File CSV creato con successo: edifici_solarolo_costruzione.csv")
        else:
            print("Errore: Impossibile trovare la tabella degli edifici per data di costruzione.")
    else:
        print("Errore: Impossibile trovare la sezione 'Gli edifici a Solarolo per data di costruzione'.")
    
    return df

topici = [f"Gli edifici a {comune} per data di costruzione", f"Gli edifici a {comune} per numero di piani", f"Gli edifici a {comune} per numero di interni",
          "Redditi e contribuenti per tipo di reddito", "Redditi, imposte e addizionali comunali e regionali", "Redditi e contribuenti per fasce di reddito"]

for x in topici:
    print(x)
    display(table(x))

Gli edifici a Civita per data di costruzione


,Date,Prima del 1919,1919-45,1946-60,1961-70,1971-80,1981-90,1991-2000,2001-05,Dopo il 2005,Comune
0,Edifici,403,97,28,11,24,7,3,3,0,Civita


Gli edifici a Civita per numero di piani


,Numero di piani,Uno,Due,Tre,Quattro o piÃ¹,Comune
0,Edifici,181,347,46,2,Civita


Gli edifici a Civita per numero di interni


...

,Categoria,Contribuenti,Reddito,Media annuale,Media mensile,Anno precedente,Variazione,Comune
0,Reddito da fabbricati,286,€ 134.155,"€ 469,07","€ 39,09","€ 32,29","€ 6,80",Civita
1,Reddito da lavoro dipendente,244,€ 3.367.680,"€ 13.801,97","€ 1.150,16","€ 1.159,14","€ -8,98",Civita
2,Reddito da pensione,325,€ 3.412.873,"€ 10.501,15","€ 875,10","€ 844,21","€ 30,89",Civita
3,Reddito da lavoro autonomo,10,€ 203.083,"€ 20.308,30","€ 1.692,36","€ 1.216,60","€ 475,76",Civita
4,Imprenditori in cont. ordin.,0,€ 0,"€ 0,00","€ 0,00","€ 0,00","€ 0,00",Civita
5,Imprenditori in cont. sempl.,25,€ 242.956,"€ 9.718,24","€ 809,85","€ 761,24","€ 48,61",Civita
6,Redditi da partecipazione,13,€ 81.555,"€ 6.273,46","€ 522,79","€ 842,76","€ -319,97",Civita


Redditi, imposte e addizionali comunali e regionali


,Categoria,Contribuenti,Reddito,Media annuale,Media mensile,Anno precedente,Variazione,Comune
0,Reddito imponibile,624,€ 7.325.926,"€ 11.740,27","€ 978,36","€ 956,33","€ 22,02",Civita
1,Reddito imp. addizionale,362,€ 5.948.469,"€ 16.432,23","€ 1.369,35","€ 1.414,00","€ -44,65",Civita
2,Imposta netta,372,€ 907.014,"€ 2.438,21","€ 203,18","€ 209,62","€ -6,43",Civita
3,Addizionale comunale,352,€ 29.610,"€ 84,12","€ 7,01","€ 7,18","€ -0,17",Civita
4,Addizionale regionale,349,€ 102.068,"€ 292,46","€ 24,37","€ 24,90","€ -0,53",Civita


Redditi e contribuenti per fasce di reddito


,Categoria,Contribuenti,Reddito,Media annuale,Media mensile,Anno precedente,Variazione,Comune
0,Reddito complessivo minore di zero euro,0,€ 0,"€ 0,00","€ 0,00","€ 0,00","€ 0,00",Civita
1,Reddito complessivo da 0 a 10.000 euro,336,€ 1.701.503,"€ 5.064,00","€ 422,00","€ 429,65","€ -7,65",Civita
2,Reddito complessivo da 10.000 a 15.000 euro,113,€ 1.362.143,"€ 12.054,36","€ 1.004,53","€ 988,47","€ 16,06",Civita
3,Reddito complessivo da 15.000 a 26.000,123,€ 2.448.245,"€ 19.904,43","€ 1.658,70","€ 1.668,61","€ -9,91",Civita
4,Reddito complessivo da 26.000 a 55.000 euro,51,€ 1.654.002,"€ 32.431,41","€ 2.702,62","€ 2.764,13","€ -61,52",Civita
5,Reddito complessivo da 55.000 a 75.000 euro,0,€ 0,"€ 0,00","€ 0,00","€ 0,00","€ 0,00",Civita
6,Reddito complessivo da 75.000 a 120.000 euro,0,€ 0,"€ 0,00","€ 0,00","€ 0,00","€ 0,00",Civita
7,Reddito complessivo oltre 120.000 euro,0,€ 0,"€ 0,00","€ 0,00","€ 0,00","€ 0,00",Civita


## Le parrocchie presenti nel comune

In [17]:
# Trova il titolo della sezione con una ricerca flessibile
titolo_sezione = None
for tag in pagina_del_comune.find_all(["h2", "h3", "span", "p"]):  # Controlliamo più tag possibili
    if re.search(r"Parrocchie", tag.get_text(), re.IGNORECASE):
        titolo_sezione = tag
        break

if titolo_sezione:
    # Trova la tabella che segue il titolo della sezione
    tabella = titolo_sezione.find_next("table")

    if tabella:
        # Estrai le intestazioni della tabella
        intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

        # Estrai i dati
        dati = []
        for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
            colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
            dati.append(colonne)

        # Creazione del DataFrame
        df = pd.DataFrame(dati, columns=intestazioni)

        # Salva il DataFrame in un file CSV
        #df.to_csv("parrocchie_solarolo.csv", index=False, encoding="utf-8")

        #print("✅ File CSV creato con successo: parrocchie_solarolo.csv")
    else:
        print("❌ Errore: Impossibile trovare la tabella delle parrocchie.")
else:
    print("❌ Errore: Non è stata trovata la sezione 'Parrocchie'.")

df.head()

,Chiesa,Comune,Frazione,Indirizzo,Parrrocchiani,Diocesi
0,S. Maria Assunta,Civita,,Dato Mancante,1.124,Eparchia di Lungro


In [18]:
pd.concat([df.head(), df.tail()], ignore_index=True)

,Chiesa,Comune,Frazione,Indirizzo,Parrrocchiani,Diocesi
0,S. Maria Assunta,Civita,,Dato Mancante,1.124,Eparchia di Lungro
1,S. Maria Assunta,Civita,,Dato Mancante,1.124,Eparchia di Lungro


## Scuole presenti nel comune

In [19]:
# Trova la sezione delle scuole
scuole_section = pagina_del_comune.find("a", {"name": "scuole"})

if scuole_section:
    table = scuole_section.find_next("table")  # Trova la tabella successiva

    # Estrarre tutte le righe della tabella
    rows = []
    for tr in table.find_all("tr"):
        cells = [td.text.strip() for td in tr.find_all(["th", "td"])]
        rows.append(cells)

    # Se la tabella ha almeno una riga, usa la prima come intestazione
    if len(rows) > 1:
        headers = rows[0]  # Prima riga come intestazione
        data = rows[1:]     # Dati senza la prima riga
    else:
        headers = None
        data = rows

    # Creare un DataFrame Pandas
    df = pd.DataFrame(data, columns=headers if headers else None)

    # Stampare il risultato
    #print(df)
else:
    print("Tabella delle scuole non trovata.")

df.head()

,Nome,Tipo,Comune,Indirizzo,Email e Pec,Telefono e fax,CAP
0,Civita - Centro,Scuola Primaria,Civita,,csic85800t@istruzione.it,098173060,87010
